In [ ]:
import torch
from PIL import Image
from transformers import Blip2Processor
from src.models.surroundblip import SurroundBlip

# 1) 사전학습된 체크포인트 지정
model_name = "Salesforce/blip2-opt-2.7b"
device = "mps" if torch.cuda.is_available() else "cpu"

# 2) 프로세서와 모델 불러오기
processor = Blip2Processor.from_pretrained(model_name)
model = SurroundBlip.from_pretrained(model_name, torch_dtype=torch.float16, cache_dir='.cache/').to(device)

# 3) 파노라마 이미지 로드 (equirectangular)
pano = Image.open("path/to/panorama.jpg").convert("RGB")

# 4) crop_equirectangular_tensor 적용을 위해 pixel_values 준비
#    — Blip2Processor 로 통상적인 center crop만 지원하므로,  
#      SurroundBlip.forward()에 (B,P,C,H,W) 형태로 넣어야 합니다.
#    아래는 예시로 연속 8방향(90° FOV, 50% 오버랩) perspective 뷰를 얻는 과정입니다.

# (a) 전처리: 전체 파노라마를 processor에 한 번에 넣어 resize
inputs = processor(images=pano, return_tensors="pt", size=(224*2,224*4)).to(device)  # (1,3,448,896)
pixel_values = inputs["pixel_values"]  # shape = (1,3,448,896)

# (b) SurroundBlip.forward 내부에서 사용하는 방식과 동일하게 (B,P,C,H,W)로 reshape
#     여기서는 직접 호출해 보기 위해 P,S,D 단계 없이 pixel_values만 넘깁니다.

# 5) 텍스트 프롬프트 설정
prompt = "Question: 이 장면에서 보이는 동물은 무엇인가요? Answer:"
text_inputs = processor(text=prompt, return_tensors="pt", padding="max_length", max_length=32).to(device)
input_ids = text_inputs["input_ids"]
attention_mask = text_inputs["attention_mask"]

# 6) 모델 인퍼런스
model.eval()
with torch.no_grad():
    outputs = model(
        pixel_values=pixel_values,          # (B, P, C, H, W) 형태를 가정 (P=1일 때도 동작)
        input_ids=input_ids,
        attention_mask=attention_mask,
        interpolate_pos_encoding=True,      # 해상도 보간 활성화
        use_cache=True
    )
    generated_ids = model.generate(
        pixel_values=pixel_values,
        input_ids=input_ids,
        max_new_tokens=16,
        num_beams=3
    )
    answer = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

print("Generated Answer:", answer)

/Users/seungwoo/anaconda3/envs/sur/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]